<a href="https://colab.research.google.com/github/Jimmynycu/Two_LoRA/blob/main/Multiple_Lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate peft bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 1.7 MB/s eta 0:00:00


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes
import accelerate
import torch
from peft import PeftModel
base_model = "meta-llama/Llama-2-7b-chat-hf"
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model, device_map={"": 0},  quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
def generate(prompt):
  tokenized_input = tokenizer(prompt, return_tensors="pt")
  input_ids = tokenized_input["input_ids"].cuda()

  generation_output = model.generate(
          input_ids=input_ids,
          num_beams=1,
          return_dict_in_generate=True,
          output_scores=True,
          max_new_tokens=130

  )
  for seq in generation_output.sequences:
      output = tokenizer.decode(seq, skip_special_tokens=True)
      print(output.strip())

In [8]:
model = PeftModel.from_pretrained(model, "kaitchup/Llama-2-7B-oasstguanaco-adapter", adapter_name="oasst")#.to('cpu')
model.load_adapter("kaitchup/Llama-2-7b-mt-French-to-English", adapter_name="fren")

adapter_config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/92.9M [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/92.9M [00:00<?, ?B/s]

_IncompatibleKeys(missing_keys=['base_model.model.model.embed_tokens.weight', 'base_model.model.model.layers.0.self_attn.q_proj.weight', 'base_model.model.model.layers.0.self_attn.k_proj.weight', 'base_model.model.model.layers.0.self_attn.v_proj.weight', 'base_model.model.model.layers.0.self_attn.o_proj.weight', 'base_model.model.model.layers.0.mlp.gate_proj.base_layer.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_A.oasst.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_B.oasst.weight', 'base_model.model.model.layers.0.mlp.up_proj.base_layer.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_A.oasst.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_B.oasst.weight', 'base_model.model.model.layers.0.mlp.down_proj.base_layer.weight', 'base_model.model.model.layers.0.mlp.down_proj.lora_A.oasst.weight', 'base_model.model.model.layers.0.mlp.down_proj.lora_B.oasst.weight', 'base_model.model.model.layers.0.input_layernorm.weight', 'base_model.model.m

In [9]:
model.add_weighted_adapter(["fren", "oasst"], [1.0,1.0], combination_type="cat", adapter_name="fren_oasst")

In [10]:
model.delete_adapter("fren")
model.delete_adapter("oasst")
model.save_pretrained("./cat_1_1/")

/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:528: UserWarning: Adapter oasst was active which is now deleted. Setting active adapter to fren_oasst.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

In [11]:
model = PeftModel.from_pretrained(model, "./cat_1_1/fren_oasst")

In [12]:
#Test generation with a translation prompt
generate("Tu es le seul client du magasin. ###>")

#Test generation with an oasst prompt
generate("### Human: Hello!### Assistant:")

Tu es le seul client du magasin. ###>You're the only customer in the store.
### Human: Hello!### Assistant: Hello! How can I assist you today? Is there something you would like to chat about or a task you would like me to help you with?
